In [18]:

!pip install -q torch transformers sentence-transformers textwrap3 flashtext nltk scikit-learn similarity
!python -m nltk.downloader all
!pip install sense2vec==2.0.0 spacy==3.5.0
!pip install git+https://github.com/boudinfl/pke.git
!pip install similarity
!pip install strsimpy
!pip install spacy==3.5.0
!python -m spacy download en_core_web_sm
import warnings
warnings.filterwarnings("ignore")

import os
import random
import time
import numpy as np
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
from sentence_transformers import SentenceTransformer
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords, wordnet as wn
from sklearn.metrics.pairwise import cosine_similarity
from strsimpy.normalized_levenshtein import NormalizedLevenshtein
import spacy
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from nltk.corpus import wordnet as wn


from flashtext import KeywordProcessor
import string
import pke
import nltk
import pickle
import spacy
nlp = spacy.load("en_core_web_sm")
nltk.download('wordnet')
nltk.download('omw-1.4')


nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if device == "cuda":
        torch.cuda.manual_seed_all(seed)

def postprocesstext(content):
    final = ""
    for sent in sent_tokenize(content):
        sent = sent.capitalize()
        final += " " + sent
    return final.strip()

def summarizer(text, model, tokenizer):
    text = "summarize: " + text.strip().replace("\n", " ")
    max_len = 512
    encoding = tokenizer.encode_plus(
        text, max_length=max_len, pad_to_max_length=False, truncation=True, return_tensors="pt"
    ).to(device)
    input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]
    outs = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        early_stopping=True,
        num_beams=3,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        min_length=75,
        max_length=300,
    )
    summary = [tokenizer.decode(ids, skip_special_tokens=True) for ids in outs][0]
    return postprocesstext(summary)

def get_nouns_multipartite(content):
    out = []
    try:

        extractor = pke.unsupervised.MultipartiteRank()
        extractor.load_document(input=content, language='en')


        pos = {'PROPN', 'NOUN'}


        stoplist = list(string.punctuation) + stopwords.words('english')


        extractor.candidate_selection(pos=pos)


        extractor.candidate_weighting(alpha=1.1, threshold=0.75, method='average')


        keyphrases = extractor.get_n_best(n=15)
        out = [val[0] for val in keyphrases]
    except Exception as e:
        print(f"Error: {e}")
    return out


def get_question(context, answer, model, tokenizer):
    text = f"context: {context} answer: {answer}"
    encoding = tokenizer.encode_plus(
        text, max_length=384, pad_to_max_length=False, truncation=True, return_tensors="pt"
    ).to(device)
    input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]
    outs = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        early_stopping=True,
        num_beams=5,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        max_length=72,
    )
    return tokenizer.decode(outs[0], skip_special_tokens=True).replace("question:", "").strip()

def mmr(doc_embedding, word_embeddings, words, top_n, lambda_param):
    word_doc_similarity = cosine_similarity(word_embeddings, doc_embedding)
    word_similarity = cosine_similarity(word_embeddings)
    keywords_idx = [np.argmax(word_doc_similarity)]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)
        mmr = lambda_param * candidate_similarities - (1 - lambda_param) * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)
    return [words[idx] for idx in keywords_idx]

from nltk.corpus import wordnet as wn
from sklearn.metrics.pairwise import cosine_similarity

def get_distractors_with_embeddings(word, origsentence, sentence_model, top_n=5):

    distractors = []
    try:

        word_embedding = sentence_model.encode([word])
        vocab = sentence_model.get_vocab()


        similar_words = []
        for vocab_word in vocab:
            vocab_embedding = sentence_model.encode([vocab_word])
            similarity = cosine_similarity(word_embedding, vocab_embedding)[0][0]
            similar_words.append((vocab_word, similarity))


        similar_words = sorted(similar_words, key=lambda x: x[1], reverse=True)
        distractors = [item[0] for item in similar_words if item[0].lower() != word.lower()][:top_n]
    except Exception as e:
        print(f"Error generating distractors with embeddings: {e}")
    return distractors

def get_distractors_with_wordnet(word, pos_tag="n", top_n=5):

    distractors = set()
    for synset in wn.synsets(word, pos=pos_tag):
        for lemma in synset.lemmas():
            lemma_name = lemma.name().replace("_", " ").lower()
            if lemma_name != word.lower():
                distractors.add(lemma_name)
    return list(distractors)[:top_n]

from nltk.corpus import wordnet as wn
import random

def get_distractors(word, question, top_n=5, lambda_val=0.2):

    distractors = []


    for syn in wn.synsets(word):
        for lemma in syn.lemmas():
            distractor = lemma.name().replace('_', ' ').lower()
            if distractor != word.lower() and distractor not in distractors:
                distractors.append(distractor)
            if len(distractors) >= top_n:
                break
        if len(distractors) >= top_n:
            break


    if len(distractors) < top_n:
        distractors += [word] * (top_n - len(distractors))

    return distractors



def get_mca_questions(context):

    summarized_text = summarizer(context, summary_model, summary_tokenizer)
    imp_keywords = get_nouns_multipartite(context)
    questions = []

    for answer in imp_keywords:

        question = get_question(summarized_text, answer, question_model, question_tokenizer)


        distractors = get_distractors(answer.capitalize(), question, top_n=5)


        if not distractors:
            distractors = imp_keywords


        distractors = [d for d in distractors if d.lower() != answer.lower()]


        options = list(set(distractors + imp_keywords))
        options = random.sample(options, 4)


        if answer.capitalize() not in options:
            options[random.randint(0, len(options) - 1)] = answer.capitalize()


        random.shuffle(options)


        questions.append({
            "question": question,
            "options": options,
            "answer": answer.capitalize(),
        })

    return questions







summary_model = T5ForConditionalGeneration.from_pretrained("t5-base").to(device)
summary_tokenizer = T5Tokenizer.from_pretrained("t5-base")
question_model = T5ForConditionalGeneration.from_pretrained("ramsrigouthamg/t5_squad_v1").to(device)
question_tokenizer = T5Tokenizer.from_pretrained("ramsrigouthamg/t5_squad_v1")
sentence_transformer_model = SentenceTransformer("sentence-transformers/msmarco-distilbert-base-v2")





/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package aver

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
context = "Automobili Lamborghini, the illustrious Italian manufacturer of luxury sports cars and SUVs, is headquartered in the picturesque Sant'Agata Bolognese. This renowned automotive institution boasts a storied legacy, and its contemporary success is firmly underpinned by a fascinating history that has seen it evolve through ownership changes, economic downturns, and groundbreaking innovations.\
Ferruccio Lamborghini, a prominent Italian industrialist with a passion for automobiles, laid the foundation for this iconic marque in 1963. His vision was audacious - to challenge the supremacy of Ferrari, the undisputed titan of Italian sports cars. Under Ferruccio's guidance, Automobili Ferruccio Lamborghini S.p.A. was established, and it immediately began making waves in the automotive world.\
One of the hallmarks of Lamborghini's early years was its distinctive rear mid-engine, rear-wheel-drive layout. This design philosophy became synonymous with Lamborghini's commitment to creating high-performance vehicles. The company's inaugural models, such as the 350 GT, arrived in the mid-1960s and showcased Lamborghini's dedication to precision engineering and uncompromising quality.\
Lamborghini's ascendancy was nothing short of meteoric during its formative decade. It consistently pushed the boundaries of automotive technology and design. However, the heady days of growth were met with a sudden downturn when the world faced the harsh realities of the 1973 global financial crisis and the subsequent oil embargo. Lamborghini, like many other automakers, grappled with plummeting sales and financial instability.\
Ownership of Lamborghini underwent multiple transitions in the wake of these challenges. The company faced bankruptcy in 1978, marking a turbulent chapter in its history. The ownership baton changed hands several times, with different entities attempting to steer the storied brand to calmer waters.\
In 1987, American automaker Chrysler Corporation took the helm at Lamborghini. The Chrysler era saw Lamborghini continue to produce remarkable vehicles like the Diablo while operating under the umbrella of a global conglomerate. However, it was not a permanent arrangement.\
In 1994, Malaysian investment group Mycom Setdco and Indonesian group V'Power Corporation acquired Lamborghini, signaling another phase of transformation for the company. These new custodians brought fresh perspectives and investment to the brand, fueling its resurgence.\
A significant turning point occurred in 1998 when Mycom Setdco and V'Power sold Lamborghini to the Volkswagen Group, which placed the Italian marque under the stewardship of its Audi division. This move brought newfound stability and resources, ensuring Lamborghini's enduring presence in the luxury sports car arena.\
Over the ensuing years, Lamborghini witnessed remarkable expansions in its product portfolio. The V10-powered Huracán captured the hearts of sports car enthusiasts with its exquisite design and formidable performance. Simultaneously, Lamborghini ventured into the SUV market with the Urus, a groundbreaking vehicle powered by a potent twin-turbo V8 engine. This diversification allowed Lamborghini to cater to a broader range of customers without compromising on its commitment to luxury and performance.\
While these successes were noteworthy, Lamborghini was not immune to the challenges posed by global economic fluctuations. In the late 2000s, during the worldwide financial crisis and the subsequent economic downturn, Lamborghini's sales experienced a significant decline, illustrating the brand's vulnerability to external economic factors.\
Despite these challenges, Lamborghini maintained its relentless pursuit of automotive excellence. The company's flagship model, the V12-powered Aventador, reached the pinnacle of automotive engineering and design before concluding its production run in 2022. However, the story does not end here. Lamborghini is set to introduce the Revuelto, a V12/electric hybrid model, in 2024, exemplifying its commitment to embracing cutting-edge technologies and pushing the boundaries of performance.\
In addition to its road car production, Lamborghini has made notable contributions to other industries. The company manufactures potent V12 engines for offshore powerboat racing, further underscoring its prowess in high-performance engineering.\
Interestingly, Lamborghini's legacy extends beyond the realm of automobiles. Ferruccio Lamborghini founded Lamborghini Trattori in 1948, a separate entity from the automobile manufacturer, which continues to produce tractors to this day.\
Lamborghini's rich history is also intertwined with the world of motorsport. In a stark contrast to his rival Enzo Ferrari, Ferruccio Lamborghini decided early on not to engage in factory-supported racing, considering it too expensive and resource-intensive. Nonetheless, Lamborghini's engineers, many of whom were passionate about racing, embarked on ambitious projects, including the development of the iconic Miura sports coupe, which possessed racing potential while being road-friendly. This project marked a pivotal moment in Lamborghini's history, showcasing its ability to create vehicles that could excel on both the track and the road.Despite Ferruccio's reluctance, Lamborghini did make some forays into motorsport. In the mid-1970s, while under the management of Georges-Henri Rossetti, Lamborghini collaborated with BMW to develop and manufacture 400 cars for BMW, a venture intended to meet Group 4 homologation requirements. However, due to financial instability and delays in development, BMW eventually took control of the project, finishing it without Lamborghini's involvement.\
Lamborghini also briefly supplied engines to Formula One teams from 1989 to 1993. Teams like Larrousse, Lotus, Ligier, Minardi, and Modena utilized Lamborghini power units during this period. Lamborghini's best result in Formula One was achieved when Aguri Suzuki finished third at the 1990 Japanese Grand Prix.\
In addition to Formula One, Lamborghini was involved in other racing series. Notably, racing versions of the Diablo were developed for the Diablo Supertrophy, a single-model racing series that ran from 1996 to 1999. The Murciélago R-GT, a production racing car, was created to compete in events like the FIA GT Championship and the American Le Mans Series in 2004, achieving notable results in its racing endeavors.\
Lamborghini's connection with motorsport reflects the brand's commitment to engineering excellence, even though it shied away from factory-backed racing for much of its history.\
Beyond the realms of automotive engineering, Lamborghini has carved a distinct niche in the world of branding. The company licenses its prestigious brand to manufacturers who produce a wide array of Lamborghini-branded consumer goods, including scale models, clothing, accessories, bags, electronics, and even laptop computers. This strategic approach has enabled Lamborghini to extend its brand reach beyond the confines of the automotive industry.\
One fascinating aspect of Lamborghini's identity is its deep connection with the world of bullfighting. In 1962, Ferruccio Lamborghini visited the ranch of Don Eduardo Miura, a renowned breeder of Spanish fighting bulls. Impressed by the majestic Miura animals, Ferruccio decided to adopt a raging bull as the emblem for his burgeoning automaker. This emblem, now iconic, symbolizes Lamborghini's passion for performance, power, and the thrill of the chase.\
Lamborghini's vehicle nomenclature also reflects this bullfighting heritage, with many models bearing the names of famous fighting bulls or bull-related themes. The Miura, named after the Miura bulls, set the precedent, and subsequent models like the Murciélago, Gallardo, and Aventador continued this tradition.\
Furthermore, Lamborghini has enthusiastically embraced emerging automotive technologies, responding to environmental concerns and changing consumer preferences. The Sian, introduced as the company's first hybrid model, showcases Lamborghini's commitment to sustainable performance. With its innovative hybrid powertrain, the Sian combines electric propulsion with a naturally aspirated V12 engine to deliver breathtaking performance while minimizing emissions.\
Looking ahead, Lamborghini has ambitious plans to produce an all-electric vehicle, aligning with the broader industry trend towards electrification. While traditionalists may lament the absence of roaring V12 engines, Lamborghini recognizes the importance of evolving with the times, ensuring that future generations of enthusiasts can experience the thrill of a Lamborghini while contributing to a more sustainable future.\
In summary, Automobili Lamborghini stands as a testament to the enduring allure of Italian craftsmanship and automotive excellence. From its audacious beginnings as a challenger to Ferrari, Lamborghini has weathered storms, embraced innovation, and left an indelible mark on the world of sports cars. Its legacy is one of design brilliance, relentless pursuit of power, and a commitment to pushing the boundaries of what's possible in the realm of high-performance automobiles. Whether through its iconic V12-powered supercars, groundbreaking hybrids, or electrifying visions of the future, Lamborghini continues to captivate the hearts of automotive enthusiasts worldwide, cementing its status as a legendary and iconic brand."
questions = get_mca_questions(context)
for q in questions:
    print(f"Question: {q['question']}")
    for i, option in enumerate(q['options']):
        print(f"{chr(65 + i)}. {option}")
    print(f"Answer: {q['answer']}\n")

Question: What is the name of the illustrious italian manufacturer of luxury sports cars?
A. testament
B. precision engineering
C. manufacturer
D. Automobili lamborghini
Answer: Automobili lamborghini

Question: What is the illustrious italian manufacturer of luxury sports cars and suvs?
A. future
B. performance
C. brand
D. Lamborghini
Answer: Lamborghini

Question: Automobili lamborghini is an illustrious what?
A. summary
B. Manufacturer
C. luxury sports cars
D. ferruccio lamborghini
Answer: Manufacturer

Question: Automobili lamborghini is known for producing what kind of cars?
A. Luxury sports cars
B. ferruccio lamborghini
C. summary
D. manufacturer
Answer: Luxury sports cars

Question: What is a summary of Lamborghini's history?
A. precision engineering
B. models
C. Summary
D. sum-up
Answer: Summary

Question: What does Lamborghini stand for?
A. automobili lamborghini
B. performance
C. future
D. Brand
Answer: Brand

Question: What is the illustrious italian manufacturer of luxury s

In [ ]:
#use this to pass api to get input
app = Flask(__name__)
from flask import Flask, request, jsonify
@app.route('/generate-mca-questions', methods=['POST'])
def generate_mca_questions():

    data = request.get_json()
    context = data.get("context")

    if not context:
        return jsonify({"error": "No context provided"}), 400


    questions = get_mca_questions(context)


    return jsonify({"questions": questions})
if __name__ == '__main__':
    app.run(debug=True)